# Creating User Defines Procedures in Snowpark:

Operationalize UDFs and UDTFs in Snowpark

1. Create UDFs from Files

2. Write Python Functions to create UDF and UDTF's

3. Register UDFs and UDTFs

4. Data types (type hints vs registration API)

For More follow links:

1. [Creating User-Defined Functions (UDFs) for DataFrames in Python](https://docs.snowflake.com/en/developer-guide/snowpark/python/creating-udfs)

2. [UDFs in Snowflake Snowpark](https://thinketl.com/udfs-in-snowflake-snowpark/)

3. [Operationalizing Snowpark Python: Part One](https://medium.com/snowflake/operationalizing-snowpark-python-part-one-892fcb3abba1)



In [ ]:
from snowflake.snowpark.functions import udf, udtf, col, lit, call_udf
from snowflake.snowpark.types import IntegerType, StructType, StructField
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.use_database("snowpark_db")
session.use_schema("sourced")
session.query_tag = "create-udfs"

df = session.create_dataframe(
    [[1], [2], [3]],
    schema=["a"])
df

In [ ]:
# anonymous scalar UDF (with udf + lambda)
add_1 = udf(
    lambda x: x+1,
    input_types=[IntegerType()],
    return_type=IntegerType())

df.select(add_1(col("a")).alias("res"))

In [ ]:
# named UDF (with udf + lambda)
add_2 = udf(
    lambda x: x+2,
    input_types=[IntegerType()],
    return_type=IntegerType(),
    name="add_2",
    replace=True)

session.sql("select add_2(1) as res")

In [ ]:
# registered UDF (with @udf decorator)
@udf(name="add_3",
    replace=True,
    is_permanent=True,
    stage_location="@stage1")
def add_3(x: int) -> int:
    return x+3

df.select(call_udf("add_3", col("a")).alias("res"))

In [ ]:
# registered UDF (with register + inline lambda)
session.udf.register(
    lambda x: x+4,
    name="add_4",
    input_types=[IntegerType()],
    return_type=IntegerType(),
    replace=True,
    is_permanent=True,
    stage_location="@stage1")

df.select(call_udf("add_4", col("a")).alias("res"))

In [ ]:
# registered UDF (from uploaded local Python file)
# must upload demo17.py file in this notebook!
add_5 = session.udf.register_from_file(
    file_path="demo17.py",
    func_name="add_5")

df.select(add_5(col("a")).alias("res"))

In [ ]:
# registered UDF (from staged Python file)
# must upload demo17.py file in this notebook!
session.file.put("demo17.py", "@stage1", auto_compress=False)

add_5 = session.udf.register_from_file(
    file_path="@stage1/demo17.py",
    func_name="add_5",
    return_type=IntegerType(),
    input_types=[IntegerType()])

df.select(add_5(col("a")).alias("res"))

In [ ]:
# registered UDTF (from implementation class)
class Get2:
    def process(self, n):
        yield(1, )
        yield(n, )

get_2 = udtf(Get2,
    input_types=[IntegerType()],
    output_schema=StructType([StructField("number", IntegerType())]))

session.table_function(get_2(lit(3)))